In [ ]:
# Decorator 
# Iterator 
# Generator
# COntext manager


In [2]:
class Vector:
    def __init__(self , x , y):
        self.x , self.y = x , y
    def __str__(self):
        return f'x is {self.x} , y is {self.y}'
    def __add__(self,another):
        return Vector(self.x+another.x , self.y+another.y)
    def __call__(self, *args, **kwds):
        return f'This object contains {self.x} , {self.y}'

v1 = Vector(10,20)
v2 = Vector(13,20)
print(v1+v2)
print(v1())

x is 23 , y is 40
This object contains 10 , 20


In [11]:
#########################
#Other Way
#########################
class SentenceIterator:
    def __init__(self, sentence):
        self.__words=sentence.split(' ')
        self.index = -1
    def __next__(self):
        self.index += 1
        try:
            return self.__words[self.index]
        except:
            raise StopIteration
class Sentence:
    def __init__(self, sentence):
        self.__text=sentence
        self.__words=sentence.split(' ')
        self.index = -1
    def __iter__(self):
        # self.index=-1 I can either have the same class as iterator class but it should do the __next__ work
        # return self
        # Or I can pass another Iterator class as my iterator like this and that will do the iteration with __next__
        return SentenceIterator(self.__text) 
    # def __next__(self):
    #     self.index+=1
    #     try:
    #         return self.__words[self.index]
    #     except:
    #         raise StopIteration
        
    def __len__(self):
        return len(self.__words)
sent = Sentence('This is my sentence')
for x in sent:
    print(x)


        
        


This
is
my
sentence


In [19]:
# My own datatype 
class StorageConverter:
    __convertUnits = { 
        'mb':{'gb':'divide 1024','kb':'multiply 1024'} ,
        'gb':{'mb':'multiply 1024','kb': 'multiply 1024*1024'},
        'kb':{'gb': 'divide 1024*1024', 'mb':'divide 1024'}
    }
    @classmethod
    def convertUnitToUnit(self, size , unit1 , unit2):
        if unit1 == unit2:
            return size
        if unit1 not in self.__convertUnits or unit2 not in self.__convertUnits:
            return -1
        expression = self.__convertUnits[unit1][unit2].split(' ')
        if 'multiply' in expression[0]:
            return size.__mul__(eval(expression[1]))
        if 'divide' in expression[0]:
            return size.__floordiv__(eval(expression[1]))
        
class Storage:
    def __init__(self,size,unit):
        self.size=StorageConverter.convertUnitToUnit(size,unit,'mb')
        self.unit='mb'
    def __add__(self,another):
        return Storage(self.size+another.size , 'mb')
    def __str__(self):
        return f'{self.size} {self.unit}'
    def printInFormat(self,format):
        return StorageConverter.convertUnitToUnit(self.size,'mb',format)


s1 = Storage(1000 , 'gb')
s2 = Storage(10 , 'mb')
s3=s1+s2
print(s3.printInFormat('kb'))




1048586240


In [33]:
# generators are something which can produce value over sequence of time 
# it will stop the current execution and you can continue later
# but that cant be recursively used as recursion may tree up and branch up into diff paths 
# you can only use geneator in sequential manner 
# if you still want to have exaustive generator use it with while True

# def generatorFibonaci(a,b):
#     yield a+b
#     generatorFibonaci(a,b)
# gen=generatorFibonaci(1,1)
# print(next(gen))
# while True:
#     next(gen)

# correct use of exhaustive decorator
def generatorFibonaci(a,b):
    while True:
        yield a+b
        a,b=b,a+b
fibgen=generatorFibonaci(1,1)
for i in range(10):
    print(next(fibgen))




2
3
5
8
13
21
34
55
89
144


In [37]:
#iterator to give first n number and sum

class FirstN:
    def __init__(self,n):
        self.n=n
    def __iter__(self):
        self.i=0
        return self
    def __next__(self):
        if self.i<self.n:
            self.i+=1
            return self.i
        else:
            raise StopIteration
f=FirstN(10)
for x in f:
    print(x)
print(sum(f))

1
2
3
4
5
6
7
8
9
10
55


In [42]:
#this is the basic example of a context manager
# this is majorly taken care by 2 dunder methods 
with open('m.txt' , 'r') as f:
    print(f.read())
#assume if you write it as sequential code 
# it would look like this 
f = open('m.txt','r')
print(f.read())

file
file
file
file
file
file


In [53]:
class FileManager:
    def __init__(self,sFile,sMode):
        self.file=sFile
        self.mode=sMode
    def __enter__(self):
        self.fobj=open(self.file,self.mode)
        return self.fobj
    def __exit__(self, exc_type, exc_value, exc_traceback):
        self.fobj.close()
        return None
with FileManager('m.txt','r') as fm:
    
    print(fm.read())

file
file
file


In [61]:
# Context manager based file reader with generator support 

class MyFileReader:
    def __init__(self,sFile,sMode):
        self.file=sFile
        self.mode=sMode
        self.index=-1
    def __next__(self):
        self.index+=1
        return self.lines[self.index]
    def __iter__(self):
        self.index=-1
        return self
    def __enter__(self):
        self.fobj=open(self.file,self.mode)
        self.lines = list(self.fobj.read())
        print(self.lines)
        return self
    def __exit__(self, exc_type, exc_value, exc_traceback):
        self.fobj.close()
        return None
    def fileGenerator(self):
        self.__iter__()
        while self.index<len(self.lines):
            yield self.__next__()
        else:
            raise StopIteration
    
    def hasNext(self):
        if self.index<len(self.lines)-1:
            return True
        return False
with MyFileReader('m.txt','r') as fm:
    # print(dir(fm))
    fgen = fm.fileGenerator()
    while fm.hasNext():
        print(next(fgen))

['f', 'i', 'l', 'e', '\n', 'f', 'i', 'l', 'e', '\n', 'f', 'i', 'l', 'e']
f
i
l
e


f
i
l
e


f
i
l
e


In [62]:
from contextlib import contextmanager
# doing context managers without __dunders__
@contextmanager
def open_managed_file(filename):
    f = open(filename, 'w')
    try:
        yield f
    finally:
        f.close()

with open_managed_file('m.txt') as f:
    f.write('some todo...')

In [110]:
#building my own cache decorator from functools without functools
def cache(fun):
    memo = {}
    def inner(*args):
        key = args
        if len(args)==1:
            key=args[0]
        # print(memo,args)
        if key in memo:
            print(f'Cache hit {key}')
            return memo[key]
        import time
        st=time.time()
        res=fun(*args)
        end=time.time()
        print(end-st)
        memo[key] = res
        # print(memo)
        # print('pjjd')
        return res
    return inner
@cache
def fib(n):
    # print(n)
    if n<0:
        return 0
    if n in [0,1]:
        return 1
    return fib(n-1)+fib(n-2)
        

fib(25)

0.0
0.0
0.0
Cache hit 1
0.0
Cache hit 2
0.0
Cache hit 3
0.0
Cache hit 4
0.0
Cache hit 5
0.0
Cache hit 6
0.0
Cache hit 7
0.0
Cache hit 8
0.0
Cache hit 9
0.0
Cache hit 10
0.0
Cache hit 11
0.0
Cache hit 12
0.0
Cache hit 13
0.0
Cache hit 14
0.0
Cache hit 15
0.0
Cache hit 16
0.0
Cache hit 17
0.0
Cache hit 18
0.0
Cache hit 19
0.0
Cache hit 20
0.0
Cache hit 21
0.0
Cache hit 22
0.0
Cache hit 23
0.0


121393

In [109]:
#timing printer using decorator
def tracktime(func):
    def wrapper(*args):
        import time
        st=time.time()
        res = func(*args)
        end=time.time()
        print(end-st)
        return res
    return wrapper
@tracktime
def foronelooptuple():
    tup = (x for x in range(0,1000000000))
    for x in tup:
        w=x
    return 1
@tracktime
def foronelooplist():
    arr = [x for x in range(0,1000000000)]
    for x in arr:
        w=x
    return 1

foronelooptuple()
foronelooplist()
    
    

71.32317781448364
684.6124761104584


1

In [117]:
#map,reduce,filter
from functools import reduce
arr=[1,2,3,4,5]
print(list(map(lambda x: x*2, arr)))
print(list(filter(lambda x: x%2, arr)))
# reduce is usually to take 2 different elements and compress it to one 
reduce(lambda x,y: x*y, arr)


[2, 4, 6, 8, 10]
[1, 3, 5]


120

In [146]:

#creational
#Singleton
class SingletonLogger:
    _instance=None
    __initialized = None
    def __init__(self,file):
        if self.__initialized == None:
            self.file=file
            self.__initialized=True
    def __new__(self,*args,**kwargs):
        if self._instance == None:
            self._instance = super().__new__(self)
        return self._instance
s1=SingletonLogger('file1')
s2=SingletonLogger('file2')
s1.file




'file1'

In [148]:
#Factory , Its like an interface to create objects and it abstracts the implementation inside the induviduals

class Animal:
    def speak(self):
        pass

class Dog(Animal):
    def speak(self):
        return "Woof!"

class Cat(Animal):
    def speak(self):
        return "Meow!"

class AnimalFactory:
    @staticmethod
    def create_animal(animal_type):
        if animal_type == "dog":
            return Dog()
        elif animal_type == "cat":
            return Cat()
        return None

dog = AnimalFactory.create_animal('dog')
print(dog.speak())

Woof!


In [149]:
# adapter is to bridge many different entities to work together for a common task
# here Payment processing is the commmon task and paypal , stripe and square are three
# different entitites . Both are connected inside unifiedpaymentadapter
class PaymentProcessor:
    def process_payment(self, amount):
        pass

# Adaptee classes
class PayPal:
    def send_payment(self, amount):
        print(f"Processing payment of ${amount} through PayPal.")

class Stripe:
    def make_payment(self, amount):
        print(f"Processing payment of ${amount} through Stripe.")

class Square:
    def pay(self, amount):
        print(f"Processing payment of ${amount} through Square.")

class UnifiedPaymentAdapter(PaymentProcessor):
    def __init__(self, payment_system, system_type):
        self.payment_system = payment_system
        self.system_type = system_type

    def process_payment(self, amount):
        if self.system_type == "paypal":
            self.payment_system.send_payment(amount)
        elif self.system_type == "stripe":
            self.payment_system.make_payment(amount)
        elif self.system_type == "square":
            self.payment_system.pay(amount)
        else:
            raise ValueError(f"Unknown payment system type: {self.system_type}")

# Client code
def main():
    paypal = PayPal()
    stripe = Stripe()
    square = Square()

    paypal_adapter = UnifiedPaymentAdapter(paypal, "paypal")
    stripe_adapter = UnifiedPaymentAdapter(stripe, "stripe")
    square_adapter = UnifiedPaymentAdapter(square, "square")

    # Using the unified adapter to process payments
    paypal_adapter.process_payment(100)  # Output: Processing payment of $100 through PayPal.
    stripe_adapter.process_payment(200)  # Output: Processing payment of $200 through Stripe.
    square_adapter.process_payment(300)  # Output: Processing payment of $300 through Square.

if __name__ == "__main__":
    main()


Processing payment of $100 through PayPal.
Processing payment of $200 through Stripe.
Processing payment of $300 through Square.
